In [1]:
import numpy as np
from scipy.sparse.linalg import LinearOperator
import matplotlib.pyplot as plt

# Opérateur d'inpainting / masquage

In [2]:
M = 6
my_mask = np.array([True, False, True, False, False, True])
A = np.eye(M, dtype=int)[my_mask, :]
print('A=', A)

A= [[1 0 0 0 0 0]
 [0 0 1 0 0 0]
 [0 0 0 0 0 1]]


In [3]:
x = np.arange(M)
print('x=', x)
print('Ax=', A@x, '=', x[my_mask])

x= [0 1 2 3 4 5]
Ax= [0 2 5] = [0 2 5]


In [4]:
class MaskOperator(LinearOperator):
    def __init__(self, mask, dtype):
        shape = (np.count_nonzero(mask), mask.size)
        LinearOperator.__init__(self, dtype=dtype, shape=shape)
        self.mask = mask
    
    def _matvec(self, x):
        return x[self.mask]
    
    def _rmatvec(self, y):
        x = np.zeros(self.mask.shape, dtype=self.dtype)
        x[self.mask] = y
        return x

In [5]:
A_op = MaskOperator(mask=my_mask, dtype=int)
print(A_op @ x)
print(A_op.T @ np.array([4,5,6]))

[0 2 5]
[4 0 5 0 0 6]


In [6]:
# Vérification de la propriété de l'adjoint
A_op = MaskOperator(mask=my_mask, dtype=float)
for _ in range(20):
    x = np.random.randn(A_op.shape[1])
    y = np.random.randn(A_op.shape[0])
    p1 = np.vdot(A_op @ x, y)
    p2 = np.vdot(x, A_op.T @ y)
    print(f'{p1-p2:.1e}, {p1}, {p2}')

0.0e+00, -1.607054139381823, -1.607054139381823
0.0e+00, 2.9722307383880637, 2.9722307383880637
0.0e+00, 0.2740228128280156, 0.2740228128280156
0.0e+00, -0.8647129018802773, -0.8647129018802773
0.0e+00, 1.1726685780602948, 1.1726685780602948
0.0e+00, 1.490003872300991, 1.490003872300991
0.0e+00, -0.004743177763543617, -0.004743177763543617
0.0e+00, 1.1118740491405366, 1.1118740491405366
0.0e+00, -2.853160736823568, -2.853160736823568
0.0e+00, 0.5547802693319382, 0.5547802693319382
0.0e+00, -2.9980658353529823, -2.9980658353529823
0.0e+00, -1.396759658852035, -1.396759658852035
0.0e+00, 3.1418206313118504, 3.1418206313118504
0.0e+00, 0.8998098457433688, 0.8998098457433688
0.0e+00, -1.904106871583596, -1.904106871583596
0.0e+00, -1.6600427171173033, -1.6600427171173033
0.0e+00, 0.9569745180842906, 0.9569745180842906
0.0e+00, 1.4978773242904684, 1.4978773242904684
0.0e+00, -0.8560581944516608, -0.8560581944516608
0.0e+00, 1.8790905546594714, 1.8790905546594714


# Opérateur de convolution

In [7]:
from scipy.linalg import circulant
from scipy.ndimage import convolve1d

N = 10
h = np.random.randint(low=-9, high=9, size=N)
A = circulant(np.concatenate((h[len(h)//2:], h[:len(h)//2])))
print('A=', A)

A= [[-3 -4  1 -9  2 -9 -4  3 -1 -1]
 [-1 -3 -4  1 -9  2 -9 -4  3 -1]
 [-1 -1 -3 -4  1 -9  2 -9 -4  3]
 [ 3 -1 -1 -3 -4  1 -9  2 -9 -4]
 [-4  3 -1 -1 -3 -4  1 -9  2 -9]
 [-9 -4  3 -1 -1 -3 -4  1 -9  2]
 [ 2 -9 -4  3 -1 -1 -3 -4  1 -9]
 [-9  2 -9 -4  3 -1 -1 -3 -4  1]
 [ 1 -9  2 -9 -4  3 -1 -1 -3 -4]
 [-4  1 -9  2 -9 -4  3 -1 -1 -3]]


In [8]:
x = np.arange(N)
print('x=', x)
print('Ax=', A@x, '=', convolve1d(x, h, mode='wrap'))

x= [0 1 2 3 4 5 6 7 8 9]
Ax= [ -86 -101 -116 -171 -156  -91 -136  -71 -106  -91] = [ -86 -101 -116 -171 -156  -91 -136  -71 -106  -91]


In [9]:
class ConvOperator(LinearOperator):
    def __init__(self, h):
        LinearOperator.__init__(self, dtype=h.dtype, shape=(h.size, h.size))
        self.h = h

        if h.size % 2:
            h_adjoint = h[::-1]
        else:
            h_adjoint = np.zeros_like(h)
            h_adjoint[0] = h[0]
            h_adjoint[1:] = h[-1:0:-1]
        self.h_adjoint = h_adjoint
    
    def _matvec(self, x):
        return convolve1d(x, self.h, mode='wrap')
    
    def _rmatvec(self, y):
        return convolve1d(y, self.h_adjoint, mode='wrap')

In [10]:
A_op = ConvOperator(h=h)
print(A_op @ x)

[ -86 -101 -116 -171 -156  -91 -136  -71 -106  -91]


In [11]:
# Vérification de la propriété de l'adjoint
for N in (8,9):
    print('N =', N)
    h = np.random.randn(N)
    A_op = ConvOperator(h=h)
    for _ in range(20):
        x = np.random.randn(A_op.shape[1])
        y = np.random.randn(A_op.shape[0])
        p1 = np.vdot(A_op @ x, y)
        p2 = np.vdot(x, A_op.T @ y)
        print(f'{p1-p2:.1e}, {p1}, {p2}')

N = 8
1.8e-15, 5.106236081532656, 5.106236081532654
-4.4e-16, -3.6991221378480765, -3.699122137848076
-2.2e-16, -1.6744224621049413, -1.674422462104941
-1.1e-16, 0.8613325622775502, 0.8613325622775503
0.0e+00, 7.182364123458639, 7.182364123458639
0.0e+00, -5.441325946519644, -5.441325946519644
7.8e-16, 0.5846239584437367, 0.5846239584437359
0.0e+00, 2.768337181883358, 2.768337181883358
0.0e+00, 0.46811511384278165, 0.46811511384278165
0.0e+00, 7.260503266971265, 7.260503266971265
-5.6e-16, -0.9099918632056611, -0.9099918632056605
0.0e+00, 2.2292563398806857, 2.2292563398806857
0.0e+00, -4.712088730702972, -4.712088730702972
5.6e-17, 0.46967986426196356, 0.4696798642619635
6.7e-16, 0.9508913157427731, 0.9508913157427724
2.2e-16, -1.0198930982054615, -1.0198930982054617
8.9e-16, -1.9638121618589055, -1.9638121618589064
-1.8e-15, -9.999165938738415, -9.999165938738413
0.0e+00, 1.1526880045870582, 1.1526880045870582
-4.4e-16, 2.7289090688909763, 2.7289090688909767
N = 9
1.8e-15, -9.3102613